In [65]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

movies = pd.read_csv(r'D:\Users\vika1\Downloads\movies.csv')
ratings = pd.read_csv(r'D:\Users\vika1\Downloads\ratings.csv')

In [67]:
movies.drop(['genres'], axis = 1, inplace = True)
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [68]:
ratings.drop(['timestamp'], axis = 1, inplace = True)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [118]:
user_item_matrix = ratings.pivot(index = 'movieId', columns = 'userId', values = 'rating')
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
user_item_matrix.fillna(0, inplace = True)
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
users_votes = ratings.groupby('userId')['rating'].agg('count')
movies_votes = ratings.groupby('movieId')['rating'].agg('count')

user_filter = users_votes[users_votes > 50].index
movies_filter = movies_votes[movies_votes > 10].index

user_item_matrix = user_item_matrix.loc[movies_filter,:]
user_item_matrix = user_item_matrix.loc[:,user_filter]
user_item_matrix.shape

(2121, 378)

In [75]:
csr_data = csr_matrix(user_item_matrix.values)
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,movieId,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [78]:
knn = NearestNeighbors(metric = 'cosine', algorithm='brute', n_neighbors = 20, n_jobs = -1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [116]:
recommendations = 10
search_word = input('What movie did you like? ')

What movie did you like? Lord of the Rings


In [117]:
movie_search = movies[movies['title'].str.contains(search_word)]

movie_id = movie_search.iloc[0] ['movieId']
movie_id = user_item_matrix [user_item_matrix['movieId'] == movie_id].index[0]

distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = recommendations + 1)
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()
distances_indices = list(zip(indices_list, distances_list))
distances_indices_sorted = sorted(distances_indices, key = lambda x: x[1], reverse = False)
distances_indices_sorted = distances_indices_sorted[1:]

recommend_list = []
for distances_indices in distances_indices_sorted:
    matrix_movie_id = user_item_matrix.iloc[distances_indices[0]]['movieId']
    id = movies[movies['movieId'] == matrix_movie_id].index
    title = movies.iloc[id]['title'].values[0]
    dist = distances_indices[1]
    recommend_list.append({'Title' : title, 'Distance' : dist})
    
recomend_df = pd.DataFrame(recommend_list, index = range(1, recommendations + 1))
recomend_df

,Title,Distance
1,Excalibur (1981),0.434872
2,Young Sherlock Holmes (1985),0.518432
3,Ladyhawke (1985),0.528053
4,Conan the Barbarian (1982),0.552975
5,Planet of the Apes (1968),0.562688
6,Superman II (1980),0.572544
7,Big Trouble in Little China (1986),0.573421
8,Logan's Run (1976),0.575343
9,Invasion of the Body Snatchers (1956),0.575582
10,Bottle Rocket (1996),0.578306
